# 使用 Unsloth 对 DeepSeek-R1-Distill-Qwen-1.5B 模型进行 LoRA 微调

本 Notebook 展示了如何使用 `unsloth` 库对 `deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B` 模型进行高效的 QLoRA (Low-Rank Adaptation) 微调。

整个流程包括：
1.  环境准备与库导入
2.  加载预训练模型和分词器 (Tokenizer)。
3.  在微调前，对模型进行简单的推理测试。
4.  下载和格式化训练数据集
5.  使用 `unsloth` 的 `FastLanguageModel` 来为模型添加 LoRA 适配器。
6.  配置 `SFTTrainer` 监督微调训练配置。
7.  启动训练，并观察 Loss 变化情况
8.  保存微调后的模型
9.  测试训练后的生成结果

### 1. 环境准备与库导入

首先，我们需要安装并导入所有必要的库。`transformers` 用于加载模型和分词器，`unsloth` 用于高效微调，`trl` 提供了 `SFTTrainer`，而 `datasets` 用于处理数据。

**注意**: 在运行此 Notebook 之前，请确保已安装所有依赖包：
```bash
pip install -r requirements.txt
```

In [1]:
import os
os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"

In [1]:
import torch
from unsloth import FastLanguageModel
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, GenerationConfig, DataCollatorForSeq2Seq
from datasets import Dataset

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


/home/sherry/miniconda3/envs/unsloth_new/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth Zoo will now patch everything to make training faster!


### 2. 加载预训练模型和分词器 (Tokenizer)

In [2]:
# 定义模型和一些基本参数
max_seq_length = 8192
dtype = None # None 表示自动选择 (Float16 a T4, V100, BFloat16 a Ampere)
load_in_4bit = True # 使用 4bit 量化加载

# 这是您的模型标识符，请替换为您正在使用的模型
# 例如："qwen-1.5b_lora_model"
# model_name = "qwen-1.5b_lora_model" 
# model_name = "unsloth/DeepSeek-R1-Distill-Qwen-1.5B" 
model_name = "unsloth/DeepSeek-R1-Distill-Qwen-1.5B-unsloth-bnb-4bit" 

# 这一步会返回一个经过 Unsloth 优化的模型和一个分词器
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

==((====))==  Unsloth 2026.2.1: Fast Qwen2 patching. Transformers: 4.57.6.
   \\   /|    NVIDIA GeForce RTX 5070 Ti Laptop GPU. Num GPUs = 1. Max memory: 11.94 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.10.0+cu128. CUDA: 12.0. CUDA Toolkit: 12.8. Triton: 3.6.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.34. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


### 3. 微调前推理测试

在对模型进行任何修改之前，我们先用它来生成一段文本，看看原始模型的表现如何。这可以作为我们微调效果的基准参考。

In [3]:
# 模型推理的 Prompt 模板
inference_prompt = """以下是一条描述任务的指令，并配有一个提供进一步上下文的输入。
请撰写一份恰当的回复，以完成该请求。
在回答之前，请仔细思考该问题，并构建一个分步的思考过程，以确保回应的逻辑严谨和内容准确。


### Instruction:
你是一位医学专家，在临床推理、诊断学和治疗规划方面拥有深厚的专业知识。
请回答以下医学问题。

### Question:
{}

### Response:
<think>{}
"""

In [4]:
FastLanguageModel.for_inference(model)

question = "男，28岁，程序员，最近一周每天工作到半夜，感觉头晕、脖子疼，有时候还恶心。"

inputs = tokenizer([inference_prompt.format(question, "")], return_tensors="pt").to("cuda")
attention_mask = inputs.input_ids.ne(tokenizer.pad_token_id).long().to("cuda")

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1200,
    use_cache=True,
)

In [5]:
response = tokenizer.batch_decode(outputs, skip_special_tokens=True)

In [6]:
print(response[0].split("### Response:")[1])


<think>
好的，我现在需要处理这个医学问题。让我先仔细阅读一下问题描述。

用户是28岁的男性，程序员，最近一周每天工作到半夜，感觉头晕、脖子疼，有时候还恶心。他需要我作为医学专家，提供合适的诊断和治疗方案。

首先，头晕和脖子疼可能是脑力损伤导致的，但也不一定，也有可能是神经损伤。需要考虑是否有脑力损伤或者神经系统的问题，比如脑力损伤可能与脑力劳动有关，但需要进一步检查确认。

接下来，恶心和呕吐可能与胃肠道功能障碍有关，比如胃炎、胃食管反流病等，但也可以是消化系统功能受损，需要进一步评估。

另外，工作时间到半夜，可能与脑力劳动相关，但可能也与神经损伤有关，需要综合考虑。

接下来，我需要考虑是否有脑力损伤或者神经损伤，可能需要进行脑力损伤评估，如果确实如此，可能需要药物治疗或神经康复。如果没有，可能需要考虑胃肠道功能障碍。

此外，工作时间到半夜可能与神经损伤相关，比如中风或脑梗死的可能性，需要进一步评估。

可能还需要考虑是否有其他因素，比如睡眠不足、压力、焦虑等，这些都可能影响脑力功能。

综合来看，我需要建议用户提供进一步的临床评估，包括脑力损伤评估、神经功能评估、胃肠道功能评估等，以便做出准确的诊断和治疗方案。
</think>

鉴于您的描述，建议您提供进一步的临床评估，包括脑力损伤评估、神经功能评估、胃肠道功能评估等，以便做出准确的诊断和治疗方案。


---

### 4. 下载和格式化训练数据集


医学推理数据集：https://huggingface.co/datasets/FreedomIntelligence/medical-o1-reasoning-SFT/viewer/zh

![dataset](images/dataset.png)

In [7]:
# 模型训练的 Prompt 模板
train_prompt = """以下是一条描述任务的指令，并配有一个提供进一步上下文的输入。
请撰写一份恰当的回复，以完成该请求。
在回答之前，请仔细思考该问题，并构建一个分步的思考过程，以确保回应的逻辑严谨和内容准确。


### Instruction:
你是一位医学专家，在临床推理、诊断学和治疗规划方面拥有深厚的专业知识。
请回答以下医学问题。

### Question:
{}

### Response:
<think>
{}
</think>
{}
"""

In [8]:
EOS_TOKEN = tokenizer.eos_token # 添加 EOS Token

def formatting_prompts_func(examples):
    inputs = examples["Question"]
    cots = examples["Complex_CoT"]
    outputs = examples["Response"]
    texts = []
    for input, cot, output in zip(inputs, cots, outputs):
        # 将 EOS Token 添加到样本最后
        text = train_prompt.format(input, cot, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset
dataset = load_dataset("FreedomIntelligence/medical-o1-reasoning-SFT", "zh", split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True,)

In [9]:
dataset[0]["text"]

'以下是一条描述任务的指令，并配有一个提供进一步上下文的输入。\n请撰写一份恰当的回复，以完成该请求。\n在回答之前，请仔细思考该问题，并构建一个分步的思考过程，以确保回应的逻辑严谨和内容准确。\n\n\n### Instruction:\n你是一位医学专家，在临床推理、诊断学和治疗规划方面拥有深厚的专业知识。\n请回答以下医学问题。\n\n### Question:\n根据描述，一个1岁的孩子在夏季头皮出现多处小结节，长期不愈合，且现在疮大如梅，溃破流脓，口不收敛，头皮下有空洞，患处皮肤增厚。这种病症在中医中诊断为什么病？\n\n### Response:\n<think>\n这个小孩子在夏天头皮上长了些小结节，一直都没好，后来变成了脓包，流了好多脓。想想夏天那么热，可能和湿热有关。才一岁的小孩，免疫力本来就不强，夏天的湿热没准就侵袭了身体。\n\n用中医的角度来看，出现小结节、再加上长期不愈合，这些症状让我想到了头疮。小孩子最容易得这些皮肤病，主要因为湿热在体表郁结。\n\n但再看看，头皮下还有空洞，这可能不止是简单的头疮。看起来病情挺严重的，也许是脓肿没治好。这样的情况中医中有时候叫做禿疮或者湿疮，也可能是另一种情况。\n\n等一下，头皮上的空洞和皮肤增厚更像是疾病已经深入到头皮下，这是不是说明有可能是流注或瘰疬？这些名字常描述头部或颈部的严重感染，特别是有化脓不愈合，又形成通道或空洞的情况。\n\n仔细想想，我怎么感觉这些症状更贴近瘰疬的表现？尤其考虑到孩子的年纪和夏天发生的季节性因素，湿热可能是主因，但可能也有火毒或者痰湿造成的滞留。\n\n回到基本的症状描述上看，这种长期不愈合又复杂的状况，如果结合中医更偏重的病名，是不是有可能是涉及更深层次的感染？\n\n再考虑一下，这应该不是单纯的瘰疬，得仔细分析头皮增厚并出现空洞这样的严重症状。中医里头，这样的表现可能更符合‘蚀疮’或‘头疽’。这些病名通常描述头部严重感染后的溃烂和组织坏死。\n\n看看季节和孩子的体质，夏天又湿又热，外邪很容易侵入头部，对孩子这么弱的免疫系统简直就是挑战。头疽这个病名听起来真是切合，因为它描述的感染严重，溃烂到出现空洞。\n\n不过，仔细琢磨后发现，还有个病名似乎更为合适，叫做‘蝼蛄疖’，这病在中医里专指像这种严重感染并伴有深部空洞的情况。它也涵盖了化脓和皮肤增厚这些症状。\n\n

In [10]:
from IPython.display import display, Markdown

display(Markdown(dataset[0]["text"]))

以下是一条描述任务的指令，并配有一个提供进一步上下文的输入。
请撰写一份恰当的回复，以完成该请求。
在回答之前，请仔细思考该问题，并构建一个分步的思考过程，以确保回应的逻辑严谨和内容准确。


### Instruction:
你是一位医学专家，在临床推理、诊断学和治疗规划方面拥有深厚的专业知识。
请回答以下医学问题。

### Question:
根据描述，一个1岁的孩子在夏季头皮出现多处小结节，长期不愈合，且现在疮大如梅，溃破流脓，口不收敛，头皮下有空洞，患处皮肤增厚。这种病症在中医中诊断为什么病？

### Response:
<think>
这个小孩子在夏天头皮上长了些小结节，一直都没好，后来变成了脓包，流了好多脓。想想夏天那么热，可能和湿热有关。才一岁的小孩，免疫力本来就不强，夏天的湿热没准就侵袭了身体。

用中医的角度来看，出现小结节、再加上长期不愈合，这些症状让我想到了头疮。小孩子最容易得这些皮肤病，主要因为湿热在体表郁结。

但再看看，头皮下还有空洞，这可能不止是简单的头疮。看起来病情挺严重的，也许是脓肿没治好。这样的情况中医中有时候叫做禿疮或者湿疮，也可能是另一种情况。

等一下，头皮上的空洞和皮肤增厚更像是疾病已经深入到头皮下，这是不是说明有可能是流注或瘰疬？这些名字常描述头部或颈部的严重感染，特别是有化脓不愈合，又形成通道或空洞的情况。

仔细想想，我怎么感觉这些症状更贴近瘰疬的表现？尤其考虑到孩子的年纪和夏天发生的季节性因素，湿热可能是主因，但可能也有火毒或者痰湿造成的滞留。

回到基本的症状描述上看，这种长期不愈合又复杂的状况，如果结合中医更偏重的病名，是不是有可能是涉及更深层次的感染？

再考虑一下，这应该不是单纯的瘰疬，得仔细分析头皮增厚并出现空洞这样的严重症状。中医里头，这样的表现可能更符合‘蚀疮’或‘头疽’。这些病名通常描述头部严重感染后的溃烂和组织坏死。

看看季节和孩子的体质，夏天又湿又热，外邪很容易侵入头部，对孩子这么弱的免疫系统简直就是挑战。头疽这个病名听起来真是切合，因为它描述的感染严重，溃烂到出现空洞。

不过，仔细琢磨后发现，还有个病名似乎更为合适，叫做‘蝼蛄疖’，这病在中医里专指像这种严重感染并伴有深部空洞的情况。它也涵盖了化脓和皮肤增厚这些症状。

哦，该不会是夏季湿热，导致湿毒入侵，孩子的体质不能御，其病情发展成这样的感染？综合分析后我觉得‘蝼蛄疖’这个病名真是相当符合。
</think>
从中医的角度来看，你所描述的症状符合“蝼蛄疖”的病症。这种病症通常发生在头皮，表现为多处结节，溃破流脓，形成空洞，患处皮肤增厚且长期不愈合。湿热较重的夏季更容易导致这种病症的发展，特别是在免疫力较弱的儿童身上。建议结合中医的清热解毒、祛湿消肿的治疗方法进行处理，并配合专业的医疗建议进行详细诊断和治疗。
<｜end▁of▁sentence｜>

### 5. 使用 Unsloth 添加 LoRA 适配器

这是使用 `unsloth` 的核心步骤。我们调用 `FastLanguageModel.get_peft_model`，它会非常高效地为模型注入 LoRA 模块。

- `r`: LoRA 的秩 (rank)，是控制模型复杂度和参数量的关键超参数。
- `target_modules`: 指定要在哪些线性层（如注意力机制的 q, k, v, o 投影层）上应用 LoRA。
- `lora_alpha`: LoRA 的缩放因子，通常设置为 `r` 的两倍或与 `r` 相同。
- `use_gradient_checkpointing`: 一种节省显存的技术，对于训练大模型至关重要。

In [11]:
# 因为 `model` 对象现在是由 Unsloth 创建的，它包含了所有必需的属性
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=[
      "q_proj",
      "k_proj",
      "v_proj",
      "o_proj",
      "gate_proj",
      "up_proj",
      "down_proj",
    ],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=1432,
    use_rslora=False,
    loftq_config=None,
)
# 检查模型结构，确认 LoRA 适配器已添加
print(model)

Unsloth 2026.2.1 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Qwen2ForCausalLM(
      (model): Qwen2Model(
        (embed_tokens): Embedding(151936, 1536, padding_idx=151654)
        (layers): ModuleList(
          (0): Qwen2DecoderLayer(
            (self_attn): Qwen2Attention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=1536, out_features=1536, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=1536, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=1536, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear(
       

### 6. 配置 SFTTrainer

`SFTTrainer` (Supervised Fine-tuning Trainer) 是一个专门用于指令微调的训练器。我们需要配置 `TrainingArguments` 来指定所有的训练参数，如批量大小、学习率、优化器等。

In [ ]:
from trl import SFTConfig, SFTTrainer
# 第一次微调
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    packing = False, # Can make training 5x faster for short sequences.
    args = SFTConfig(
        per_device_train_batch_size = 4,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 60,
        learning_rate = 2e-4,
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 1432,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

In [ ]:
from trl import SFTConfig, SFTTrainer
# 第二次微调
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    packing = False, # Can make training 5x faster for short sequences.
    args = SFTConfig(
        per_device_train_batch_size = 4,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = 2, # Set this for 1 full training run.
        max_steps = 60,
        learning_rate = 1e-4,
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 1432,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

In [16]:
!nvidia-smi
import torch
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))



Sun Feb 15 09:13:45 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 590.52.01              Driver Version: 591.74         CUDA Version: 13.1     |
+-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 5070 ...    On  |   00000000:01:00.0  On |                  N/A |
| N/A   42C    P4             23W /  115W |    3240MiB /  12227MiB |      1%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

### 7. 开始训练

一切准备就绪后，调用 `trainer.train()` 即可开始微调过程。训练结束后，会返回包含训练统计信息（如训练损失）的对象。

In [17]:
trainer_stats = trainer.train()

# 打印训练统计信息
print(trainer_stats)

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 20,171 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 4 x 1) = 16
 "-____-"     Trainable parameters = 18,464,768 of 1,795,552,768 (1.03% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,2.292900
2,2.325600
3,2.305600
4,2.273200
5,2.265100
6,2.220000
7,2.222200
8,2.241500
9,2.277300
10,2.185100


TrainOutput(global_step=60, training_loss=2.2185691316922505, metrics={'train_runtime': 3853.7724, 'train_samples_per_second': 0.249, 'train_steps_per_second': 0.016, 'total_flos': 6882985986097152.0, 'train_loss': 2.2185691316922505, 'epoch': 0.04759071980963712})


<div>  
      <progress value='60' max='60' style='width:300px; height:20px; vertical-align: middle;'></progress>
      [60/60 11:32, Epoch 0/1]
    </div>
    <table border="1" class="dataframe">
  <thead>
 <tr style="text-align: left;">
      <th>Step</th>
      <th>Training Loss</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>1</td>
      <td>3.217100</td>
    </tr>
    <tr>
      <td>2</td>
      <td>3.261000</td>
    </tr>
    <tr>
      <td>3</td>
      <td>3.203700</td>
    </tr>
    <tr>
      <td>4</td>
      <td>3.199200</td>
    </tr>
    <tr>
      <td>5</td>
      <td>3.166700</td>
    </tr>
    <tr>
      <td>6</td>
      <td>2.965400</td>
    </tr>
    <tr>
      <td>7</td>
      <td>2.979000</td>
    </tr>
    <tr>
      <td>8</td>
      <td>2.936400</td>
    </tr>
    <tr>
      <td>9</td>
      <td>2.902400</td>
    </tr>
    <tr>
      <td>10</td>
      <td>2.773200</td>
    </tr>
    <tr>
      <td>11</td>
      <td>2.803000</td>
    </tr>
    <tr>
      <td>12</td>
      <td>2.913300</td>
    </tr>
    <tr>
      <td>13</td>
      <td>2.762400</td>
    </tr>
    <tr>
      <td>14</td>
      <td>2.738200</td>
    </tr>
    <tr>
      <td>15</td>
      <td>2.465800</td>
    </tr>
    <tr>
      <td>16</td>
      <td>2.767100</td>
    </tr>
    <tr>
      <td>17</td>
      <td>2.620200</td>
    </tr>
    <tr>
      <td>18</td>
      <td>2.360900</td>
    </tr>
    <tr>
      <td>19</td>
      <td>2.456200</td>
    </tr>
    <tr>
      <td>20</td>
      <td>2.462800</td>
    </tr>
    <tr>
      <td>21</td>
      <td>2.446300</td>
    </tr>
    <tr>
      <td>22</td>
      <td>2.450500</td>
    </tr>
    <tr>
      <td>23</td>
      <td>2.259400</td>
    </tr>
    <tr>
      <td>24</td>
      <td>2.272000</td>
    </tr>
    <tr>
      <td>25</td>
      <td>2.326700</td>
    </tr>
    <tr>
      <td>26</td>
      <td>2.335800</td>
    </tr>
    <tr>
      <td>27</td>
      <td>2.384300</td>
    </tr>
    <tr>
      <td>28</td>
      <td>2.301100</td>
    </tr>
    <tr>
      <td>29</td>
      <td>2.440700</td>
    </tr>
    <tr>
      <td>30</td>
      <td>2.438200</td>
    </tr>
    <tr>
      <td>31</td>
      <td>2.327600</td>
    </tr>
    <tr>
      <td>32</td>
      <td>2.337000</td>
    </tr>
    <tr>
      <td>33</td>
      <td>2.535900</td>
    </tr>
    <tr>
      <td>34</td>
      <td>2.278300</td>
    </tr>
    <tr>
      <td>35</td>
      <td>2.297000</td>
    </tr>
    <tr>
      <td>36</td>
      <td>2.313000</td>
    </tr>
    <tr>
      <td>37</td>
      <td>2.273700</td>
    </tr>
    <tr>
      <td>38</td>
      <td>2.104500</td>
    </tr>
    <tr>
      <td>39</td>
      <td>2.370900</td>
    </tr>
    <tr>
      <td>40</td>
      <td>2.473200</td>
    </tr>
    <tr>
      <td>41</td>
      <td>2.251000</td>
    </tr>
    <tr>
      <td>42</td>
      <td>2.256500</td>
    </tr>
    <tr>
      <td>43</td>
      <td>2.106900</td>
    </tr>
    <tr>
      <td>44</td>
      <td>2.072500</td>
    </tr>
    <tr>
      <td>45</td>
      <td>2.318200</td>
    </tr>
    <tr>
      <td>46</td>
      <td>2.214300</td>
    </tr>
    <tr>
      <td>47</td>
      <td>2.355900</td>
    </tr>
    <tr>
      <td>48</td>
      <td>2.345700</td>
    </tr>
    <tr>
      <td>49</td>
      <td>2.278200</td>
    </tr>
    <tr>
      <td>50</td>
      <td>2.285500</td>
    </tr>
    <tr>
      <td>51</td>
      <td>2.218000</td>
    </tr>
    <tr>
      <td>52</td>
      <td>2.268200</td>
    </tr>
    <tr>
      <td>53</td>
      <td>2.159700</td>
    </tr>
    <tr>
      <td>54</td>
      <td>2.362600</td>
    </tr>
    <tr>
      <td>55</td>
      <td>2.151900</td>
    </tr>
    <tr>
      <td>56</td>
      <td>2.369500</td>
    </tr>
    <tr>
      <td>57</td>
      <td>2.236100</td>
    </tr>
    <tr>
      <td>58</td>
      <td>2.194500</td>
    </tr>
    <tr>
      <td>59</td>
      <td>2.276700</td>
    </tr>
    <tr>
      <td>60</td>
      <td>2.340200</td>
    </tr>
  </tbody>
</table><p>

### 8. 保存微调后的模型（Lora）

训练完成后，您可以再次进行推理，比较微调后的模型与原始模型的差异。如果对结果满意，可以使用 `model.save_pretrained("your_lora_adapter_path")` 来保存训练好的 LoRA 适配器。

In [19]:
model.save_pretrained("qwen-1.5b_lora_model")

In [20]:
tokenizer.save_pretrained("qwen-1.5b_lora_model")

('qwen-1.5b_lora_model/tokenizer_config.json',
 'qwen-1.5b_lora_model/special_tokens_map.json',
 'qwen-1.5b_lora_model/chat_template.jinja',
 'qwen-1.5b_lora_model/tokenizer.json')

In [ ]:
# 模型保存方式二选一（要么使用上面的分开保存，要么使用这里的合并 Lora 保存）
# model.save_pretrained_merged("qwen-1.5b_lora_model", tokenizer, save_method="merged_16bit")

### 9. 测试训练后的生成结果

In [21]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

question="一个患有急性阑尾炎的病人已经发病5天，腹痛稍有减轻但仍然发热，在体检时发现右下腹有压痛的包块，此时应如何处理？", # Question
inputs = tokenizer([inference_prompt.format(question, "")], return_tensors="pt").to("cuda")

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1000,
    use_cache=True,
)

In [22]:
output = tokenizer.batch_decode(outputs, skip_special_tokens=True)
print(output[0].split("### Response:")[1])


<think>
这个病人已经急性阑尾炎，5天了，腹痛稍微减轻，但还是有发热。体检发现右下腹有个压痛的包块。嗯，这个包块可能不是大问题，但让我仔细想想。

首先，压痛包块可能和阑尾炎有关。阑尾炎是右下腹的常见病，通常导致压痛包块，但得注意不要弄错了。压痛包块可能会导致感染，所以得注意预防。

再想想，这个病人已经病程长了，虽然症状没有太严重，但不能忽视。我们得用积极的态度，看看有没有其他可能。

比如，这个病人可能有其他并发症，如感染。所以，要特别注意，不能忽视。

另外，从病程来看，5天的病程是较短的，但要保持警惕，确保没有大问题。

如果发现包块，要立刻去看外科医生，不要犹豫。毕竟，包块的大小和位置可能影响治疗。

此外，要避免让病人继续在医院里，因为可能会导致感染，这样更安全。

总的来说，要立刻处理，但也要注意不要拖延，必要的话，再找个专家看看。

嗯，总的来说，处理这个压痛包块应该立即，但也要有预防意识。
</think>
对于一个已经发病5天、腹痛稍有减轻但仍然发热的病人，右下腹出现压痛包块的情况，应立即进行处理。以下是一些建议：

1. **立即行动**：由于病程较短，可能需要立即处理，避免拖延，以免感染扩散。

2. **评估包块大小和位置**：观察包块的大小和位置，判断其是否需要进一步处理。

3. **排除感染的可能性**：如果包块较大，或者包块与感染有关，应立即进行感染性治疗。

4. **考虑其他并发症**：注意是否还有其他并发症，如感染或其他并发症，如感染性休克等，需采取相应措施。

5. **紧急处理**：如果包块较大或有感染迹象，应立即进行感染性处理，如抗生素治疗或手术。

6. **避免延误**：避免等待医院进一步检查，以确保及时采取措施。

7. **预防感染**：在处理包块之前，可能需要进行积极的预防措施，如抗生素治疗或使用抗感染药物。

最终，应立即进行处理，但需谨慎，以确保安全和有效性。



In [23]:
def generate_response(question: str, model, tokenizer, inference_prompt: str, max_new_tokens: int = 1024) -> str:
    """
    使用指定的模型和分词器为给定的医学问题生成响应。

    Args:
        question (str): 需要模型回答的医学问题。
        model: 已加载的 Unsloth/Hugging Face 模型。
        tokenizer: 对应的分词器。
        inference_prompt (str): 用于格式化输入的 f-string 模板。
        max_new_tokens (int, optional): 生成响应的最大 token 数量。默认为 1024。

    Returns:
        str: 模型生成的响应文本，已去除 prompt 部分。
    """
    # 1. 使用模板格式化输入
    prompt = inference_prompt.format(
        question, # 填充问题
        "",       # 留空，让模型生成 CoT 和 Response
    )

    # 2. 将格式化后的 prompt 进行分词，并转移到 GPU
    inputs = tokenizer([prompt], return_tensors="pt").to(model.device)

    # 3. 使用模型生成输出
    # use_cache=True 用于加速解码过程
    outputs = model.generate(
        input_ids=inputs.input_ids,
        attention_mask=inputs.attention_mask,
        max_new_tokens=max_new_tokens,
        use_cache=True,
    )
    
    # 4. 将生成的 token 解码为文本
    # skip_special_tokens=True 会移除像 EOS_TOKEN 这样的特殊标记
    decoded_output = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

    # 5. 切分字符串，只返回 "### Response:" 之后的部分
    # 使用 .split() 分割并获取响应内容，.strip() 用于去除可能存在的前后空白字符
    response_part = decoded_output.split("### Response:")
    if len(response_part) > 1:
        return response_part[1].strip()
    else:
        # 如果模型没有生成 "### Response:" 标记，则返回整个生成内容以供调试
        return decoded_output

In [24]:
my_question = "对于一名60岁男性患者，出现右侧胸疼并在X线检查中显示右侧肋膈角消失，诊断为肺结核伴右侧胸腔积液，请问哪一项实验室检查对了解胸水的性质更有帮助？"

response = generate_response(my_question, model, tokenizer, inference_prompt)
print("==================== 模型回答 ====================")
print(response)

==================== 模型回答 ====================
<think>
患者60岁，有右胸痛，X线检查显示右肋膈角消失，这是肺结核的典型表现。肺结核通常会导致肺部结核病，导致肺部积液，特别是胸腔积液。这种情况下，胸腔中的液体可能形成积液，尤其是当肺部结核病导致肺部压力增大时。

所以，要了解这种积液的性质，比如是否是液体、固体还是其他形态，这需要对胸腔的液体进行分析。一种常用的检验方法是血常规，它可以帮助我们了解血液中各种血细胞和血蛋白的含量，从而判断胸腔中的液体类型。

再考虑，血常规可以帮助我们了解血液中的血细胞和血蛋白，从而判断胸腔中的液体是否是液体、固体或其他形态。比如，如果血蛋白的水平升高，通常说明有胸腔积液，而血细胞的含量变化则可能提示有气体积聚。此外，血常规还能帮助我们判断胸腔中的液体是否是液体，而没有其他形态。

因此，血常规是针对了解胸腔积液性质的最优选择。
</think>
在对右胸腔积液进行诊断时，血常规是评估胸腔液体性质的重要工具。它能够帮助我们了解血液中血细胞和血蛋白的含量，从而判断胸腔中的液体类型。例如，如果血蛋白水平升高，通常表明有胸腔积液。此外，血细胞的含量变化也能提示有气体积聚或液体积聚。因此，血常规能够帮助我们确定胸腔中的液体是液体，而不是其他形态，从而更好地理解胸腔积液的性质。


In [25]:
my_question = "对于一名 28 岁的男性患者，工作是程序员，常年熬夜，最近突然感觉头晕目眩，甚至有点恶心。请问有可能是什么疾病？"

response = generate_response(my_question, model, tokenizer, inference_prompt)
print("==================== 模型回答 ====================")
print(response)

==================== 模型回答 ====================
<think>
患者是一个28岁的男性，他工作是程序员，一直熬夜，最近突然感觉头晕目眩，还有点恶心。这让我想到可能是什么问题呢？

嗯，头晕目眩，还有恶心，这很可能是脑力负担导致的。程序员长时间工作，可能就是这样的问题。

哦，对了，脑力过载可能就是个原因。工作压力大，导致脑力活动量增加，从而影响到脑力神经节律，影响到各种感觉。

嗯，脑力过载可能是最直接的原因，再加上脑力活动量增加，可能会影响脑力神经节律，进而导致头晕目眩和恶心。

不过，也有可能是脑部功能异常，比如脑部功能减退或损伤。这可能是因为长时间的工作导致脑部功能减退，进而影响到各种感觉和认知功能。

嗯，脑部功能异常也可能是个可能性，但我觉得脑力过载更直接一点。

再想想，可能还有其他原因，比如脑力活动量增加导致脑力神经节律改变，进而影响到各种感觉。

嗯，脑力过载，脑力神经节律改变，这些都是可能的原因。

哦，对了，脑力神经节律改变可能是因为长时间的脑力活动量增加，进而影响到各种感觉。

嗯，我感觉这些因素都可能，但脑力过载是最直接的。

所以，综合来看，脑力过载可能是最可能的原因。
</think>
根据患者的症状描述，头晕目眩和恶心可能是由于脑力活动量增加导致的脑力神经节律改变。这种症状可能与长时间的工作压力有关，特别是程序员长期熬夜工作，可能导致脑力活动量增加，从而影响到脑力神经节律，进而导致各种感觉。

脑力过载是脑力活动量增加的一种常见现象，这种情况下，脑部功能可能受到严重影响，导致脑力神经节律改变。因此，脑力过载可能是患者症状的主要原因。

然而，也有可能脑部功能异常，如脑部功能减退或损伤，这可能与脑力活动量增加有关。脑部功能异常可能影响到各种感觉和认知功能，但脑力过载的直接原因更为明确。

综上所述，脑力过载可能是患者症状的主要原因，但也需要考虑脑部功能异常的可能性。建议进一步进行脑部功能评估，以确认其原因。
